In [29]:
# System libraries
import logging, os, sys
import gzip
import re
import sqlite3

In [32]:
db_path = '../Data/input/schema_org_reviews.db'
schema_path = '../Data/input/schema_Review.gz'
bestworstratinglog_path='../Logs/bestWorstRating.log'

# Enable logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename=bestworstratinglog_path, mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [33]:
i               = 0
skipped_lines   = 0
inserted        = 0
not_taxo        = 0
failed_updates  = 0

print(db_path)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

../Data/input/schema_org_reviews.db


In [36]:
taxo_pattern = re.compile("<http://schema.org/aggregaterating/bestRating>|<http://schema.org/aggregaterating/worstRating>", re.IGNORECASE)
split_pattern = re.compile("^(_:.*)\s(<http:\/\/schema.org\/aggregaterating\/bestRating>|<http:\/\/schema.org\/aggregaterating\/worstRating>)\s(.*)\s<(.*)>\s\.$", re.IGNORECASE)


with gzip.open(schema_path,"rt") as f:
    # head = [next(f) for x in range(500)]
    for line in iter(f.readline,""):
        i += 1
        if not taxo_pattern.search(line):
            not_taxo += 1
            continue
        match = split_pattern.match(line)
        if match is None:
            skipped_lines += 1
            continue
        props = match.groups()
        with conn:
            subj = props[0]
            pred = props[1]
            obj = props[2]
            source = props[3]
            conn.execute("INSERT OR IGNORE INTO REVIEWS (NODE,URL) VALUES (?,?)",(subj,obj))
            update_query = "UPDATE REVIEWS SET "
            params = [obj, subj, source]
            if pred.lower() == "<http://schema.org/aggregaterating/bestRating>".lower():
                update_query += "BESTRATING = ?"
            if pred.lower() == "<http://schema.org/aggregaterating/worstRating>".lower():
                update_query += "WORSTRATING = ?"
            update_query += " WHERE NODE = ? AND URL = ?;"
            try:
                conn.execute(update_query, params)
                inserted += 1
            except:
                print("failed to execute for params " + str(props))
                failed_updates += 1

out1 = ("inserted " + str(inserted) + " lines and skipped " + str(skipped_lines) + " lines and not in chosen taxonomy " + str(not_taxo))
out2 = ("Done processing the review file")
logging.debug(out1)
logging.debug(out2)

